In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u252-b09-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

In [6]:

# create the Spark Context
sc = spark.sparkContext

## EJ 4 Se tiene un RDD con las coordenadas de rectángulos de la forma (x1,x2,y1,y2). Se pide programar en PySpark un programa que encuentre el rectángulo de superficie mínima que contiene al punto (w,z)   

In [23]:
Coordenadas = [('marca1', 'modelo1', 5, 1),
 ('marca1', 'modelo1', 10 , 1),
 ('marca2', 'modelo2', 23, 1),
 ('marca1', 'modelo1', 12, 1),
('marca2', 'modelo2', 45, 1),
('marca1', 'modelo1', 8, 1)]

In [31]:
CoordenadasRDD = sc.parallelize(Coordenadas, 8)
CoordenadasRDD.collect()

[('marca1', 'modelo1', 5, 1),
 ('marca1', 'modelo1', 10, 1),
 ('marca2', 'modelo2', 23, 1),
 ('marca1', 'modelo1', 12, 1),
 ('marca2', 'modelo2', 45, 1),
 ('marca1', 'modelo1', 8, 1)]

In [32]:
Ventas = CoordenadasRDD.map(lambda x: ((x[0], x[1]), (x[2], x[3]) ) )

In [33]:
Ventas.collect()

[(('marca1', 'modelo1'), (5, 1)),
 (('marca1', 'modelo1'), (10, 1)),
 (('marca2', 'modelo2'), (23, 1)),
 (('marca1', 'modelo1'), (12, 1)),
 (('marca2', 'modelo2'), (45, 1)),
 (('marca1', 'modelo1'), (8, 1))]

In [34]:
Ventas = Ventas.reduceByKey(lambda a,b: (a[0] if a[0] < b[0] else b[0], a[0] if a[0] > b[0] else b[0],a[1]+b[1]) )
Ventas.collect()

[(('marca1', 'modelo1'), (5, 8, 13)), (('marca2', 'modelo2'), (23, 45, 2))]

In [ ]:
FiltroPunto = CoordenadasRDD.filter(lambda x: x[1]>= w and x[0]<= w and x[3]>= z and x[2]<= z )
FiltroPunto.collect()

[(5, 8, 0, 5), (4, 6, 5, 10), (2, 7, 2, 10)]

In [ ]:
Rectangulos = FiltroPunto.map(lambda x: (x[1]-x[0] ,  x[3]-x[2]) )
Rectangulos.collect()

Areas = Rectangulos.map(lambda x: (x[1]*x[0])).reduce(lambda a,b : a if a<b else b)

Areas

10

In [ ]:
Tweets = sc.parallelize([(15, 'hola que lindo que estas #cute #baby'), (9, '#paz #amor')], 2) 
Tweets.collect()


[(15, 'hola que lindo que estas #cute #baby'), (9, '#paz #amor')]

In [ ]:
publi =  Tweets.flatMap(lambda x: ((x[0], palabra) for palabra in x[1].split()))
publi.collect()


[(15, 'hola'),
 (15, 'que'),
 (15, 'lindo'),
 (15, 'que'),
 (15, 'estas'),
 (15, '#cute'),
 (15, '#baby'),
 (9, '#paz'),
 (9, '#amor')]

In [ ]:
prueba = publi.filter(lambda x:  x[1].startswith('#'))